In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import os
import numpy as np
import cv2
from tqdm import tqdm

REBUILD_DATA = False

class DogsVSCats():
    IMG_SIZE = 50
    CATS = 'cats_dogs/training_set/cats'
    DOGS = 'cats_dogs/training_set/dogs'
    TESTING = "cats_dogs/test_set"
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []
    
    print(os.listdir(CATS))
    
    catcount = 0
    dogcount = 0
    
    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for file in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, file)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)   #not necessary to convert to b/w in cnns. 3d also works!
                                                                   #always check, do you really need colour?
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))

                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])

                    if label == self.CATS:
                        self.catcount += 1

                    if label == self.DOGS:
                        self.dogcount += 1
                except Exception as e:
                    pass
                
        np.random.shuffle(self.training_data)
        np.save('training_data.npy', self.training_data)
        print('Cats:', self.catcount)
        print('Dogs:', self.dogcount)
    
if REBUILD_DATA:
    dogsvscats = DogsVSCats()
    dogsvscats.make_training_data()


training_data = np.load('drive/My Drive/training_data.npy', allow_pickle=True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__() # just run the init of parent class (nn.Module)
            
        #Convulational Layers
        self.conv1 = nn.Conv2d(1, 32, 5) # input is 1 image, 32 output channels, 5x5 kernel / window
        self.conv2 = nn.Conv2d(32, 64, 5) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 conv
        self.conv3 = nn.Conv2d(64, 128, 5)

        x = torch.randn(50,50).view(-1, 1, 50, 50)   #-1: x is a 'batch' of any size. So, if only one item => Batch of 1 item.
        self._to_linear = None
        self.convs(x)

        #Finding fully_connected layers
        self.fc1 = nn.Linear(self._to_linear, 512) #flattening.
        self.fc2 = nn.Linear(512, 2) # 512 in, 2 out bc we're doing 2 classes (dog vs cat).
            
    def convs(self, x):
        #Max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))   #first relu on conv layer, then max pool
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        
        # print(x[0].shape)
        if self._to_linear is None:
            self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]   #dimensions of input to nn.Linear
        return x   #max_pooled layer, to be passed to nn.Linear (i.e. flattening)
    
    def forward(self, x):
        x = self.convs(x)
        
        #Flattening
        x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before 
        x = F.relu(self.fc1(x))
        x = self.fc2(x) #because this is our output layer. No activation here.
        return F.softmax(x, dim=1)
    
    
net = Net()   #object of class Net
print(net)   # prints object attributes OR class Net data members
            

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X / 255.0
y = torch.Tensor([i[1] for i in training_data])

testset_percent = 0.1
testset_size = int(len(X) * testset_percent)

In [ ]:
print("Model's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

In [ ]:
train_X = X[:-testset_size]   #0 to len(train_X) - testset_size
train_y = y[:-testset_size]

test_X = X[-testset_size:]   #len(train_X) - testset_size to end
test_y = y[-testset_size:]

print(type(test_X[0]))

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
net.load_state_dict(torch.load('drive/My Drive/dogsvscats_stateDict.pt'))
net.eval()

In [ ]:
BATCH_SIZE = 100
#if you're getting memory errors, lower the batch size.

EPOCHS = 1

for epoch in range(EPOCHS):
    for i in range(0, len(train_X), BATCH_SIZE): # from 0, to the len of X, stepping BATCH_SIZE at a time.
        batch_X = train_X[i:i + BATCH_SIZE].view(-1, 1, 50, 50)
        batch_y = train_y[i:i + BATCH_SIZE]
        
        net.zero_grad()

        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch}. Loss: {loss}")

In [ ]:
trained_model = 'drive/My Drive/dogsvscats_stateDict.pt'
torch.save(net.state_dict(), trained_model)

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for i in range(len(test_X)):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50))[0]  # returns a list of lists. 
        predicted_class = torch.argmax(net_out)
        
        if predicted_class == real_class:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))

In [ ]:
path = 'drive/My Drive/deep_learning/cat3.jpg'
img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)   #not necessary to converty to b/w in cnns. 3d also works!
                                                                   #always check, do you really need colour?
img = cv2.resize(img, (50, 50))
img_arr = np.array(img)

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
img_tensor = torch.Tensor(img_arr).view(-1, 50, 50)

if torch.argmax(net(img_tensor.view(1, 1, 50, 50))) == torch.argmax(torch.tensor([1, 0])):
    print('Its a Cat!')
elif torch.argmax(net(img_tensor.view(1, 1, 50, 50))) == torch.argmax(torch.tensor([0, 1])):
    print('Its a Dog!')

In [ ]:
from torch import cuda

train_on_gpu = cuda.is_available()
print(f'Train on gpu: {train_on_gpu}')

# Number of gpus
if train_on_gpu:
    gpu_count = cuda.device_count()
    print(f'{gpu_count} gpus detected.')
    if gpu_count > 1:
        multi_gpu = True
    else:
        multi_gpu = False